# Importing Dependicies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
import pymysql.cursors
from datetime import datetime

# Importing Data

In [2]:
apld = pd.read_pickle('df_diabetes_s1.pickle')

# Data Filtering

In [3]:
# Paid Claims Only
apld = apld[apld['claim_type'].isin(['pd'])]

In [4]:
# Type 2 Diabetes Only
apld = apld[apld['disease_type'].isin(['t2'])]

In [6]:
# 4 Years of Data
apld['service_date'] = pd.to_datetime(apld['service_date'])

apld = apld[apld['service_date'].between('2019-01-01', '2022-12-31')]

# Exploratory Data Analysis 

### Data Structure

In [13]:
apld['claim_id'].head()

0    517566340
1    387782725
2    999807010
3    999807011
4    999807012
Name: claim_id, dtype: int64

In [16]:
apld['claim_id'].nunique()

152782

In [17]:
apld['claim_id'].count()

152791

In [20]:
len_min = apld['claim_id'].astype(str).str.len().min()
len_max = apld['claim_id'].astype(str).str.len().max()
print(' claim_id length min: '+str(len_min), '\n', 'claim_id length max: '+str(len_max))

 claim_id length min: 9 
 claim_id length max: 10


In [24]:
apld['dup'] = apld.duplicated('claim_id', keep=False)
apld['dup'].value_counts()

False    152773
True         18
Name: dup, dtype: int64

In [25]:
dups = apld[apld['dup'] == 1].sort_values(by=['claim_id'])

In [26]:
# 9 duplicate claim_ids
len(dups['claim_id'].unique())

9